<a href="https://colab.research.google.com/github/PriyaSenthilnathan/thyroid/blob/main/Tfinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
from datasets import load_dataset
from PIL import Image
import pandas as pd
import numpy as np

# Load dataset
ds = load_dataset("Leeyuyu/Image_DDTI")

# Folder to save images
IMG_DIR = "/content/drive/MyDrive/dataset/DDTI_images"
os.makedirs(IMG_DIR, exist_ok=True)

rows = []

for idx, item in enumerate(ds['train']):
    # Convert image to PIL if needed
    if isinstance(item['image'], np.ndarray):
        pil_img = Image.fromarray(item['image'])
    else:
        pil_img = item['image']  # already PIL

    # Save image
    img_filename = f"{idx+1}.jpg"
    img_path = os.path.join(IMG_DIR, img_filename)
    pil_img.save(img_path)

    # Save TIRADS
    rows.append({'filename': img_filename, 'tirads': item['tirads']})

# Save CSV mapping
label_csv = "/content/drive/MyDrive/dataset/DDTI_labels.csv"
df = pd.DataFrame(rows)
df.to_csv(label_csv, index=False)

print(f"✅ Saved {len(rows)} images and labels to Drive at {IMG_DIR}")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Resolving data files:   0%|          | 0/350 [00:00<?, ?it/s]

✅ Saved 349 images and labels to Drive at /content/drive/MyDrive/dataset/DDTI_images


In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from tensorflow.keras.utils import img_to_array
from tqdm import tqdm

# Paths
IMG_DIR = "/content/drive/MyDrive/dataset/DDTI_images"
LABEL_CSV = "/content/drive/MyDrive/dataset/DDTI_labels.csv"
OUTPUT_FOLDER = "/content/drive/MyDrive/dataset/DDTI_preprocessed"
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# Load CSV
df = pd.read_csv(LABEL_CSV)

def preprocess_image(img_path, target_size=(224,224)):
    # 1️⃣ Read image
    img = cv2.imread(img_path)

    # 2️⃣ Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # 3️⃣ Resize
    resized = cv2.resize(gray, target_size)

    # 4️⃣ Normalize
    normalized = resized / 255.0

    # 5️⃣ CLAHE contrast enhancement
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    enhanced = clahe.apply((normalized*255).astype(np.uint8))

    # 6️⃣ Expand dims for model input
    final = img_to_array(enhanced) / 255.0  # shape (224,224,1)

    return final, enhanced

# Preprocess all images
processed_images = []
processed_labels = []

for _, row in tqdm(df.iterrows(), total=len(df)):
    img_path = os.path.join(IMG_DIR, row['filename'])
    processed, enhanced = preprocess_image(img_path)

    processed_images.append(processed)
    processed_labels.append(row['tirads'])  # keep TIRADS for labeling later

    # Save preprocessed grayscale image
    save_path = os.path.join(OUTPUT_FOLDER, row['filename'])
    cv2.imwrite(save_path, enhanced)

processed_images = np.array(processed_images)
processed_labels = np.array(processed_labels)

print(f"✅ Preprocessing done. Images saved to {OUTPUT_FOLDER}")


100%|██████████| 349/349 [00:05<00:00, 63.68it/s]


✅ Preprocessing done. Images saved to /content/drive/MyDrive/dataset/DDTI_preprocessed


In [ ]:
def tirads_to_label(tirads):
    if str(tirads) in ['1','2','3']:
        return 0  # benign
    else:
        return 1  # malignant

binary_labels = np.array([tirads_to_label(t) for t in processed_labels])

print("Sample labels:", binary_labels[:10])


Sample labels: [1 1 1 1 1 0 1 1 1 1]


# **Vision Transformer**

Validation Accuracy: 0.8286

In [ ]:
import os
import pandas as pd
from PIL import Image as PilImage
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, models
import timm
from tqdm import tqdm

# Paths
base_path = "/content/drive/MyDrive/dataset/"
img_dir = os.path.join(base_path, "DDTI_preprocessed")
labels_file = os.path.join(base_path, "DDTI_labels.csv")


In [ ]:
# Mount Google Drive to access your dataset
from google.colab import drive
drive.mount('/content/drive')

# Define the paths to your dataset in Google Drive
# Updated paths as per your request
base_path = "/content/drive/MyDrive/dataset/"
img_dir = os.path.join(base_path, "DDTI_preprocessed")  # Folder with grayscale images
labels_file = os.path.join(base_path, "DDTI_labels.csv")  # CSV with filenames and tirads

# Verify that the directories and file exist
if not os.path.exists(img_dir):
    raise FileNotFoundError(f"Image directory not found: {img_dir}")
if not os.path.exists(labels_file):
    raise FileNotFoundError(f"Labels CSV file not found: {labels_file}")

print("Environment setup complete and data paths verified.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Environment setup complete and data paths verified.


In [ ]:
class CustomDDTI_Dataset(Dataset):
    def __init__(self, img_dir, labels_file, transform=None):
        self.img_labels = pd.read_csv(labels_file)
        self.img_dir = img_dir
        self.transform = transform

        # Correct mapping based on CSV values
        self.label_map = {
            '1': 0, '2': 0, '3': 0,        # Benign
            '4': 1, '4a': 1, '4b': 1, '4c': 1, '5': 1  # Malignant
        }

        # Map tirads to binary labels and convert to int
        self.img_labels['label'] = self.img_labels['tirads'].map(self.label_map).astype(int)

        # Optional: check for unmapped TIRADS
        if self.img_labels['label'].isna().any():
            raise ValueError("Some TIRADS values were not mapped. Check CSV and label_map.")

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_name = self.img_labels.iloc[idx, 0]
        img_path = os.path.join(self.img_dir, img_name)

        if not os.path.exists(img_path):
            raise FileNotFoundError(f"Image not found: {img_path}")

        # Open image and convert to RGB (ViT expects 3 channels)
        image = PilImage.open(img_path).convert("RGB")
        label = int(self.img_labels.iloc[idx]['label'])

        if self.transform:
            image = self.transform(image)

        return image, label


In [ ]:
# ViT expects 224x224 RGB images
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Dataset
dataset = CustomDDTI_Dataset(img_dir=img_dir, labels_file=labels_file, transform=train_transform)

# Train/val split 80/20
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
val_dataset.dataset.transform = val_transform

# Dataloaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=2)

print(f"✅ Train samples: {len(train_dataset)}, Val samples: {len(val_dataset)}")


✅ Train samples: 279, Val samples: 70


In [ ]:
# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pretrained ViT
model = timm.create_model('vit_base_patch16_224', pretrained=True)
model.head = nn.Linear(model.head.in_features, 2)  # binary classification
model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

print("✅ ViT model ready")


✅ ViT model ready


In [ ]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in tqdm(train_loader, desc=f"Training Epoch {epoch+1}"):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total

    # Validation
    model.eval()
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)

    val_acc = val_correct / val_total
    print(f"Epoch [{epoch+1}/{num_epochs}] | Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.4f} | Val Acc: {val_acc:.4f}")


Training Epoch 1: 100%|██████████| 18/18 [00:09<00:00,  1.87it/s]


Epoch [1/10] | Train Loss: 0.7195, Train Acc: 0.7742 | Val Acc: 0.8571


Training Epoch 2: 100%|██████████| 18/18 [00:09<00:00,  1.97it/s]


Epoch [2/10] | Train Loss: 0.5216, Train Acc: 0.7563 | Val Acc: 0.8571


Training Epoch 3: 100%|██████████| 18/18 [00:09<00:00,  1.96it/s]


...

Epoch [8/10] | Train Loss: 0.5136, Train Acc: 0.7778 | Val Acc: 0.8571


Training Epoch 9: 100%|██████████| 18/18 [00:09<00:00,  1.85it/s]


Epoch [9/10] | Train Loss: 0.4837, Train Acc: 0.8172 | Val Acc: 0.8571


Training Epoch 10: 100%|██████████| 18/18 [00:09<00:00,  1.84it/s]


Epoch [10/10] | Train Loss: 0.4543, Train Acc: 0.8172 | Val Acc: 0.8571


In [ ]:
# Switch model to evaluation mode
model.eval()

val_correct = 0
val_total = 0

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        val_correct += (preds == labels).sum().item()
        val_total += labels.size(0)

val_acc = val_correct / val_total
print(f"✅ Validation Accuracy: {val_acc:.4f}")


✅ Validation Accuracy: 0.8571


In [ ]:
from sklearn.metrics import classification_report

all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.numpy())

print(classification_report(all_labels, all_preds, target_names=['Benign', 'Malignant']))


              precision    recall  f1-score   support

      Benign       0.00      0.00      0.00        10
   Malignant       0.86      1.00      0.92        60

    accuracy                           0.86        70
   macro avg       0.43      0.50      0.46        70
weighted avg       0.73      0.86      0.79        70



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# **CNN**

 Final Validation Accuracy: 0.8857

In [ ]:
train_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.1, contrast=0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

dataset = CustomDDTI_Dataset(img_dir=img_dir, labels_file=labels_file, transform=train_transform)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
val_dataset.dataset.transform = val_transform

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=2)


In [ ]:
labels = dataset.img_labels['label'].values
class_counts = np.bincount(labels)
class_weights = 1. / class_counts
weights = torch.FloatTensor(class_weights).to('cuda' if torch.cuda.is_available() else 'cpu')

criterion = nn.CrossEntropyLoss(weight=weights)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.efficientnet_b0(pretrained=True)
num_features = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_features, 2)  # binary classification
model = model.to(device)


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)


In [ ]:
num_epochs = 10

best_val_acc = 0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0
    correct = 0
    total = 0

    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1} - Training"):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_loss = running_loss / total
    train_acc = correct / total

    # Validation
    model.eval()
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)

    val_acc = val_correct / val_total

    print(f"Epoch [{epoch+1}/{num_epochs}] "
          f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}")

    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "/content/drive/MyDrive/DDTI_EfficientNet_best.pth")
        print("✅ Saved best model")


Epoch 1 - Training: 100%|██████████| 18/18 [00:02<00:00,  6.16it/s]


Epoch [1/10] Train Loss: 0.7025 | Train Acc: 0.6165 | Val Acc: 0.5429
✅ Saved best model


Epoch 2 - Training: 100%|██████████| 18/18 [00:01<00:00,  9.46it/s]


Epoch [2/10] Train Loss: 0.6215 | Train Acc: 0.7168 | Val Acc: 0.6714
✅ Saved best model


Epoch 3 - Training: 100%|██████████| 18/18 [00:01<00:00,  9.57it/s]


...

Epoch [8/10] Train Loss: 0.2767 | Train Acc: 0.9391 | Val Acc: 0.8286


Epoch 9 - Training: 100%|██████████| 18/18 [00:01<00:00,  9.01it/s]


Epoch [9/10] Train Loss: 0.2254 | Train Acc: 0.9606 | Val Acc: 0.8714
✅ Saved best model


Epoch 10 - Training: 100%|██████████| 18/18 [00:01<00:00,  9.60it/s]


Epoch [10/10] Train Loss: 0.1868 | Train Acc: 0.9785 | Val Acc: 0.8857
✅ Saved best model


In [ ]:
# @title Default title text
# Load the best model
model.load_state_dict(torch.load("/content/drive/MyDrive/DDTI_EfficientNet_best.pth"))
model.eval()

val_correct = 0
val_total = 0

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        val_correct += (preds == labels).sum().item()
        val_total += labels.size(0)

final_val_acc = val_correct / val_total
print(f"✅ Final Validation Accuracy: {final_val_acc:.4f}")


✅ Final Validation Accuracy: 0.8857


# **Swin Transformer + CNN**

In [ ]:
!pip install timm

In [ ]:
import torch.nn as nn
import torch

class CNN_SwinHybrid(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        import timm
        import torchvision.models as models

        # CNN backbone (EfficientNet-B0)
        self.cnn = models.efficientnet_b0(pretrained=True)
        self.cnn.classifier = nn.Identity()  # remove old classifier

        # Swin Transformer backbone
        self.swin = timm.create_model('swin_tiny_patch4_window7_224', pretrained=True, features_only=True)

        # Use a dummy input to determine output feature size
        dummy = torch.randn(1, 3, 224, 224)
        cnn_f = torch.flatten(self.cnn(dummy), 1)
        swin_f = torch.flatten(self.swin(dummy)[-1], 1)
        in_features = cnn_f.shape[1] + swin_f.shape[1]

        self.classifier = nn.Linear(in_features, num_classes)

    def forward(self, x):
        cnn_out = torch.flatten(self.cnn(x), 1)
        swin_out = torch.flatten(self.swin(x)[-1], 1)
        combined = torch.cat([cnn_out, swin_out], dim=1)
        out = self.classifier(combined)
        return out


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN_SwinHybrid(num_classes=2).to(device)


In [ ]:
import torch
import torch.nn as nn
import numpy as np

# Assuming you have your labels as numpy array
labels = dataset.img_labels['label'].values  # your dataset from CustomDDTI_Dataset
class_counts = np.bincount(labels)
class_weights = 1. / class_counts
weights = torch.FloatTensor(class_weights).to(device)

criterion = nn.CrossEntropyLoss(weight=weights)


In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)


In [ ]:
from torch.utils.data import DataLoader, random_split

# Split dataset into train/val (e.g., 80/20)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# DataLoaders with small batch size to avoid OOM
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, num_workers=2)


In [ ]:
from tqdm import tqdm

num_epochs = 5  # adjust as needed

for epoch in range(num_epochs):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1} - Training"):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_loss = running_loss / total
    train_acc = correct / total

    # Validation
    model.eval()
    val_correct, val_total = 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)
    val_acc = val_correct / val_total

    print(f"Epoch {epoch+1} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}")


Epoch 1 - Training: 100%|██████████| 70/70 [00:06<00:00, 10.01it/s]


Epoch 1 | Train Loss: 0.8202 | Train Acc: 0.5412 | Val Acc: 0.5571


Epoch 2 - Training: 100%|██████████| 70/70 [00:07<00:00,  9.69it/s]


Epoch 2 | Train Loss: 0.8756 | Train Acc: 0.5591 | Val Acc: 0.5571


Epoch 3 - Training: 100%|██████████| 70/70 [00:07<00:00,  9.98it/s]


Epoch 3 | Train Loss: 0.8022 | Train Acc: 0.5556 | Val Acc: 0.5571


Epoch 4 - Training: 100%|██████████| 70/70 [00:06<00:00, 10.57it/s]


Epoch 4 | Train Loss: 0.8105 | Train Acc: 0.5771 | Val Acc: 0.5571


Epoch 5 - Training: 100%|██████████| 70/70 [00:07<00:00,  9.58it/s]


Epoch 5 | Train Loss: 0.8067 | Train Acc: 0.5878 | Val Acc: 0.5571


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, models
import timm
from PIL import Image
import pandas as pd
import os
from tqdm import tqdm
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [ ]:
class CustomDDTI_Dataset(Dataset):
    def __init__(self, img_dir, labels_file, transform=None):
        self.img_labels = pd.read_csv(labels_file)
        self.img_dir = img_dir
        self.transform = transform

        self.label_map = {
            '1': 0, '2': 0, '3': 0,
            '4': 1, '4a': 1, '4b': 1, '4c': 1, '5': 1
        }
        self.img_labels['label'] = self.img_labels['tirads'].map(self.label_map)

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_name = self.img_labels.iloc[idx, 0]
        img_path = os.path.join(self.img_dir, img_name)

        if not os.path.exists(img_path):
            raise FileNotFoundError(f"Image not found: {img_path}")

        image = Image.open(img_path).convert("RGB")
        label = int(self.img_labels.iloc[idx]['label'])

        if self.transform:
            image = self.transform(image)

        return image, label

# Strong augmentation for training
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.2, hue=0.1),
    transforms.RandomAffine(15),
    transforms.ToTensor()
])

# Validation transform
val_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor()
])


In [ ]:
img_dir = "/content/drive/MyDrive/dataset/DDTI_preprocessed"
labels_file = "/content/drive/MyDrive/dataset/DDTI_labels.csv"

dataset = CustomDDTI_Dataset(img_dir, labels_file, transform=train_transform)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Replace validation dataset transform
val_dataset.dataset.transform = val_transform

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, num_workers=2)


In [ ]:
class CNN_SwinHybrid(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()

        # CNN backbone: EfficientNet-B0
        self.cnn = models.efficientnet_b0(pretrained=True)
        self.cnn.classifier = nn.Identity()

        # Swin-Tiny backbone
        self.swin = timm.create_model('swin_tiny_patch4_window7_224', pretrained=True, features_only=True)

        # Compute feature size dynamically
        dummy = torch.randn(1, 3, 224, 224)
        cnn_f = torch.flatten(self.cnn(dummy), 1)
        swin_f = torch.flatten(self.swin(dummy)[-1], 1)
        combined_features = cnn_f.shape[1] + swin_f.shape[1]

        # Bottleneck + Dropout
        self.bottleneck = nn.Sequential(
            nn.Linear(combined_features, 512),
            nn.ReLU(),
            nn.Dropout(0.4)
        )
        self.classifier = nn.Linear(512, num_classes)

    def forward(self, x):
        cnn_out = torch.flatten(self.cnn(x), 1)
        swin_out = torch.flatten(self.swin(x)[-1], 1)
        combined = torch.cat([cnn_out, swin_out], dim=1)
        out = self.bottleneck(combined)
        out = self.classifier(out)
        return out


In [ ]:
model = CNN_SwinHybrid(num_classes=2).to(device)

labels_array = dataset.img_labels['label'].values
class_counts = np.bincount(labels_array)
class_weights = 1. / class_counts
weights = torch.FloatTensor(class_weights).to(device)

criterion = nn.CrossEntropyLoss(weight=weights)
optimizer = optim.AdamW(model.parameters(), lr=1e-5, weight_decay=1e-3)


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
num_epochs = 20
best_val_acc = 0
patience = 5
counter = 0

for epoch in range(num_epochs):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1} - Training"):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_loss = running_loss / total
    train_acc = correct / total

    # Validation
    model.eval()
    val_correct, val_total = 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)

    val_acc = val_correct / val_total
    print(f"Epoch {epoch+1} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}")

    # Early stopping & save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "/content/drive/MyDrive/best_cnn_swin_model.pth")
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print("Early stopping triggered")
            break


Epoch 1 - Training: 100%|██████████| 70/70 [00:08<00:00,  8.13it/s]


Epoch 1 | Train Loss: 0.8205 | Train Acc: 0.6272 | Val Acc: 0.9000


Epoch 2 - Training: 100%|██████████| 70/70 [00:07<00:00,  9.07it/s]


Epoch 2 | Train Loss: 0.6092 | Train Acc: 0.7634 | Val Acc: 0.8714


Epoch 3 - Training: 100%|██████████| 70/70 [00:09<00:00,  7.55it/s]


...

Epoch 4 | Train Loss: 0.4094 | Train Acc: 0.8315 | Val Acc: 0.8857


Epoch 5 - Training: 100%|██████████| 70/70 [00:07<00:00,  9.01it/s]


Epoch 5 | Train Loss: 0.3293 | Train Acc: 0.8710 | Val Acc: 0.7857


Epoch 6 - Training: 100%|██████████| 70/70 [00:08<00:00,  8.37it/s]


Epoch 6 | Train Loss: 0.2441 | Train Acc: 0.9140 | Val Acc: 0.8571
Early stopping triggered


In [27]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
from torchvision import transforms
from tqdm import tqdm
from PIL import Image
import timm
import os

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


Device: cpu


In [28]:
img_dir = "/content/drive/MyDrive/dataset/DDTI_preprocessed"
labels_file = "/content/drive/MyDrive/dataset/DDTI_labels.csv"

dataset = CustomDDTI_Dataset(img_dir, labels_file, transform=train_transform)

# Train-validation split
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Apply validation transform
val_dataset.dataset.transform = val_transform

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, num_workers=2)


In [29]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import timm
from torchvision import models

class CNN_SwinHybrid(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()

        # CNN backbone: EfficientNet-B0
        self.cnn = models.efficientnet_b0(pretrained=True)
        self.cnn.classifier = nn.Identity()  # remove final classifier

        # Swin-Tiny backbone
        self.swin = timm.create_model('swin_tiny_patch4_window7_224', pretrained=True, features_only=True)

        # Bottleneck + Dropout
        self.bottleneck = nn.Sequential(
            nn.Linear(1280 + 768, 512),  # CNN 1280 + Swin 768
            nn.ReLU(),
            nn.Dropout(0.4)
        )
        self.classifier = nn.Linear(512, num_classes)

    def forward(self, x):
        # CNN
        cnn_feat = self.cnn(x)                       # (B, 1280, H, W)
        cnn_feat = F.adaptive_avg_pool2d(cnn_feat, 1) # (B, 1280, 1, 1)
        cnn_out = torch.flatten(cnn_feat, 1)        # (B, 1280)

        # Swin
        swin_feat = self.swin(x)[-1]                # (B, 768, H', W')
        swin_feat = F.adaptive_avg_pool2d(swin_feat, 1) # (B, 768, 1, 1)
        swin_out = torch.flatten(swin_feat, 1)      # (B, 768)

        # Combine
        combined = torch.cat([cnn_out, swin_out], dim=1)  # (B, 2048)

        # Bottleneck + Classifier
        out = self.bottleneck(combined)
        out = self.classifier(out)
        return out


In [30]:
criterion = nn.CrossEntropyLoss()  # you can add class weights here if needed
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2)


In [31]:
scaler = torch.cuda.amp.GradScaler()  # for automatic mixed precision


/tmp/ipython-input-2647008226.py:1: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()  # for automatic mixed precision
/usr/local/lib/python3.12/dist-packages/torch/amp/grad_scaler.py:136: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


In [32]:
from tqdm import tqdm
import torch.amp as amp

best_val_acc = 0.0
patience, counter = 5, 0

# Initialize model
model = CNN_SwinHybrid(num_classes=2).to(device)

# Training loop
for epoch in range(20):
    model.train()
    running_loss, correct, total = 0, 0, 0

    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1} - Training"):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        with amp.autocast(device_type='cuda'):  # Updated mixed precision
            outputs = model(images)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_acc = correct / total
    train_loss = running_loss / total

    # Actual validation (replace with your val_loader)
    model.eval()
    val_correct, val_total = 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)

    val_acc = val_correct / val_total if val_total > 0 else 0

    print(f"Epoch {epoch+1} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}")

    # Step scheduler
    scheduler.step(val_acc)

    # Early stopping
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_cnn_swin_model.pth")
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print("Early stopping triggered!")
            break

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Epoch 1 - Training:   0%|          | 0/70 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/amp/autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
Epoch 1 - Training:   0%|          | 0/70 [00:03<?, ?it/s]


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 1 but got size 4 for tensor number 1 in the list.